In [4]:
import pyranges as pr
import pandas as pd
from cerberus.cerberus import *

In [5]:
gtf = 'tests/files/Canx.gtf' 
mode = 'tss'
dist = 50
slack = 50

In [6]:
bed = get_ends_from_gtf(gtf, 'tss', dist, slack)

## Getting ICs from GTF

In [22]:
def get_ics_from_gtf(gtf):
    """
    Get a file for each intron chain in a gtf and number them 
    based on tags indicating priority for each gene w/i the gtf 
    
    Parameters:
        gtf (str): Filename for input gtf

    Returns:
        ic (pandas DataFrame): 
    """
    
    # get basic status and appris_principal tag for each transcript
    t_df = get_transcript_ref(gtf)
    t_df = t_df[['transcript_id', 'MANE_Select', 'basic_set', 'appris_principal']]

    # get unique intron chains from gtf
    df = pr.read_gtf(gtf)
    df = get_ic(df)

    # add basic annotation, appris principal number, and gene id
    df = df.merge(t_df, on='transcript_id', how='left')

    # add number for each unique intron chain
    df = number_tss_ic_tes(df, mode='ic')

    # make coords into tuple and perform additional
    # formatting for this table
    df['ic'] = df.ic.str.split('-')
    df['ic'] = [tuple(c) for c in df.ic.tolist()]
    ic = df.copy(deep=True)
    ic.rename({'ic': 'Coordinates'},
               axis=1, inplace=True)
    ic['gene_id'] = ic.gene_id.str.split('.', n=1, expand=True)[0]
    ic['Name'] = ic['gene_id']+'_'+ic.ic_num.astype(str)
    cols = ['transcript_id', 'MANE_Select', 'basic_set', 'appris_principal',
            'gene_id', 'ic_num']
    ic.drop(cols, axis=1, inplace=True)
    
    return ic

In [23]:
df = get_ics_from_gtf(gtf)

In [24]:
df

,Chromosome,Strand,Coordinates,Name
0,chr11,-,"(50298116, 50297394)",ENSMUSG00000020368_4
1,chr11,-,"(50299128, 50298236, 50298116, 50297394, 50297...",ENSMUSG00000020368_5
2,chr11,-,"(50300897, 50299344)",ENSMUSG00000020368_2
3,chr11,-,"(50325492, 50311780, 50311603, 50310867, 50310...",ENSMUSG00000020368_1
4,chr11,-,"(50325492, 50311784, 50311603, 50310867, 50310...",ENSMUSG00000020368_3
